In [1]:
import pandas as pd
from google.cloud import bigquery
from pathlib import Path


In [2]:
client = bigquery.Client()


In [3]:
RAW_DATA_PATH = Path("../data/raw")
RAW_DATA_PATH.mkdir(parents=True, exist_ok=True)

EVENTS_SQL_PATH = Path("../sql/fact_events.sql")
PRODUCTS_SQL_PATH = Path("../sql/fact_products.sql")

with open(EVENTS_SQL_PATH, "r") as f:
    events_sql = f.read()

with open(PRODUCTS_SQL_PATH, "r") as f:
    products_sql = f.read()


In [4]:
job1 = client.query(events_sql)
events_df = job1.to_dataframe()

job2 = client.query(products_sql)
products_df = job2.to_dataframe()


In [ ]:
OUTPUT_FILE_PATH1 = RAW_DATA_PATH / "fact_events_raw.parquet"
OUTPUT_FILE_PATH2 = RAW_DATA_PATH / "fact_products_raw.parquet"
events_df.to_parquet(OUTPUT_FILE_PATH1, index = False)
products_df.to_parquet(OUTPUT_FILE_PATH2, index = False)
